In [1]:
import torch
import torch.nn as nn
import torchvision
from netCDF4 import Dataset
import numpy as np
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.optim as optim
import torch.multiprocessing as mp
import tempfile
import sys
import torch.nn.functional as F

from unet import * # from unet import UNet
# from torchsummary import summary

/home/bnguyen4/.conda/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Sat Apr 23 22:57:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000035:03:00.0 Off |                    0 |
| N/A   32C    P0    37W / 300W |   3073MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000035:04:00.0 Off |                    0 |
| N/A   

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
# def setup(rank, world_size):
#     os.environ['MASTER_ADDR'] = 'localhost'
#     os.environ['MASTER_PORT'] = '12355'

#     # initialize the process group
#     dist.init_process_group("gloo", rank=rank, world_size=world_size)

# def cleanup():
#     dist.destroy_process_group()
    
# def run_demo(demo_fn, world_size):
#     mp.spawn(demo_fn,
#              args=(world_size,),
#              nprocs=world_size,
#              join=True)

In [5]:
# data_dir = '/home/jcurtis2/hackathon_data/'
# wrf_filename = '%straining.nc' % data_dir
# ncf = Dataset(wrf_filename, "r", format="NETCDF4")

# data = ncf.variables['o3']
# data = torch.from_numpy(np.array(data))
# data = torchvision.transforms.Resize((157,157))(data)
# # data = torch.unsqueeze(data,0)
# data = torch.unsqueeze(data,1)
# data = data.repeat(1,10,1,1,1)
# print(data.shape) # num samples, channels, z, y, x

In [6]:
# Hyperparameter settings
NUM_EPOCHS = 1
BATCH_SIZE = 4

In [7]:
torch.cuda.empty_cache()
x = torch.randn(8, 10, 39, 157, 157).to(device)
# data = data.cuda()

bilinear = False
factor = 2 if bilinear else 1

In [8]:
net = UNet(10, 2).cuda()

In [9]:
!nvidia-smi

Sat Apr 23 22:57:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000035:03:00.0 Off |                    0 |
| N/A   33C    P0    51W / 300W |   4308MiB / 16384MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000035:04:00.0 Off |                    0 |
| N/A   

In [15]:
%%time
y = net(x)
y.shape

RuntimeError: CUDA out of memory. Tried to allocate 940.00 MiB (GPU 0; 15.00 GiB total capacity; 9.93 GiB already allocated; 427.75 MiB free; 10.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [11]:
# inc = DoubleConv(10,32,first_kernel=7,padding=3) # first_kernel=3: torch.Size([1, 64, 35, 153, 153])
# inc.to(device)
# down1 = Down(32, 64)
# down1.to(device)
# down2 = Down(64, 128 // factor)
# down2.to(device)
# up1 = Up(128, 64 // factor, bilinear)
# up1.to(device)
# up2 = Up(64, 32 // factor, bilinear)
# up2.to(device)
# outc = OutConv(32, 2)
# outc.to(device)

# x1 = inc(x)
# print(x1.shape) # torch.Size([8, 32, 31, 149, 149])

# x2 = down1(x1)
# print('x2', x2.shape) # torch.Size([8, 64, 11, 70, 70])

# x3 = down2(x2)
# print('x3', x3.shape) # torch.Size([8, 128, 1, 31, 31])

# y = up1(x3, x2)
# print('up1', y.shape)

# y = up2(y, x1)
# print('up2', y.shape)

# y = up2(y,x)
# print('outc', y.shape)

In [12]:
# def test_correct_size(rank, world_size):
#     setup(rank, world_size)
#     net = UNet(10,0).to(rank)
#     ddp_net = DDP(net, device_ids=[rank])

In [13]:
# run_demo(test_correct_size, world_size)

In [14]:
## Train


for epoch in range(NUM_EPOCHS):
    
    running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         inputs, labels = data
        
#         optimizer.zero_grad()
        
#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item
#         if i % 10 == 1:
#             print(running_loss)
#             running_loss = 0.0

print("Finished Training")

Finished Training
